### Project 2: Neural networks and CIFAR10, scripts

In this notebook are scripts which can be used to generate the data presented in the figures in the report.

In [1]:
# Load modules
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import cross_validate
from sklearn.metrics import confusion_matrix

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets,transforms 

from skorch import NeuralNetClassifier

from load_data import load_CIFAR
import model_architectures

## Load and prepare the CIFAR10 data

In [2]:
# Load the data
train_sliceable, test_sliceable , y_train, y_test = load_CIFAR()

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting classifier_data/cifar-10-python.tar.gz to classifier_data
Files already downloaded and verified


---
## Effect of different learning rates on model accuracy

In [ ]:
## Set parameters

# Set the model architecture
model1 = model_architectures.CustomCNN1_cifar10(n_classes = 10)
model2 = model_architectures.CustomCNN2_cifar10(n_classes = 10)

n_folds = 5    # Number of folds for the cross-validation
n_epochs = 5   # Number of epochs

# Set a range or learning rates
powers = np.arange(-5,1,0.5)
lr_range = np.power(10*np.ones(len(powers)), powers) 

In [ ]:
## Using the Adam optimizer

# Initialize accuracy vectors
acc_lr_cnn1 = np.zeros((len(lr_range), n_folds))
acc_lr_cnn2 = np.zeros((len(lr_range), n_folds))

# Train the model for different learning rates
for i in range(len(lr_range)):
    # CustomCNN1
    net_temp = NeuralNetClassifier(
        module = model1,
        batch_size = 64,
        max_epochs = n_epochs,
        criterion = nn.CrossEntropyLoss,
        optimizer = optim.Adam,
        optimizer__lr = lr_range[i]
        )
    
    cv_results_temp = cross_validate(net_temp, train_sliceable, y_train, cv=n_folds, scoring='accuracy')
    acc_lr_cnn1[i,:] = cv_results_temp['test_score']
    
    # CustomCNN2
    net_temp = NeuralNetClassifier(
        module = model2,
        batch_size = 64,
        max_epochs = n_epochs,
        criterion = nn.CrossEntropyLoss,
        optimizer = optim.Adam,
        optimizer__lr = lr_range[i]
        )
    
    cv_results_temp = cross_validate(net_temp, train_sliceable, y_train, cv=n_folds, scoring='accuracy')
    acc_lr_cnn2[i,:] = cv_results_temp['test_score']

# Save the results
np.savetxt('skorch_nn_cifar10_crossval_CustomCNN1_lr_adam.txt', acc_lr_cnn1)
np.savetxt('skorch_nn_cifar10_crossval_CustomCNN2_lr_adam.txt', acc_lr_cnn2)


In [ ]:
## Using SGD with momentum

# Manually set the momentum parameter. Choose one of [0, 0.5, 0.9, 0.99]
p = 0.99

# Initialize accuracy vectors
acc_lr_cnn1 = np.zeros((len(lr_range), n_folds))
acc_lr_cnn2 = np.zeros((len(lr_range), n_folds))

# Train the model for different learning rates
for i in range(len(lr_range)):
    # CustomCNN1
    net_temp = NeuralNetClassifier(
        module = model1,
        batch_size = 64,
        max_epochs = n_epochs,
        criterion = nn.CrossEntropyLoss,
        optimizer = optim.SGD,
        optimizer__momentum = p,
        optimizer__lr = lr_range[i]
        )
    
    cv_results_temp = cross_validate(net_temp, train_sliceable, y_train, cv=n_folds, scoring='accuracy')
    acc_lr_cnn1[i,:] = cv_results_temp['test_score']
    
    # CustomCNN2
    net_temp = NeuralNetClassifier(
        module = model2,
        batch_size = 64,
        max_epochs = n_epochs,
        criterion = nn.CrossEntropyLoss,
        optimizer = optim.SGD,
        optimizer__momentum = p,
        optimizer__lr = lr_range[i]
        )
    
    cv_results_temp = cross_validate(net_temp, train_sliceable, y_train, cv=n_folds, scoring='accuracy')
    acc_lr_cnn2[i,:] = cv_results_temp['test_score']

# Save the results (user needs to uncomment the line which corresponds to the correct value of p)
np.savetxt('skorch_nn_cifar10_crossval_CustomCNN1_lr_sgdp099.txt', acc_lr_cnn1)  # p=0.99
#np.savetxt('skorch_nn_cifar10_crossval_CustomCNN1_lr_sgdp09.txt', acc_lr_cnn1)  # p=0.9
#np.savetxt('skorch_nn_cifar10_crossval_CustomCNN1_lr_sgdp05.txt', acc_lr_cnn1)  # p=0.5
#np.savetxt('skorch_nn_cifar10_crossval_CustomCNN1_lr_sgdp0.txt', acc_lr_cnn1)   # p=0

np.savetxt('skorch_nn_cifar10_crossval_CustomCNN2_lr_sgdp099.txt', acc_lr_cnn2)  # p=0.99
#np.savetxt('skorch_nn_cifar10_crossval_CustomCNN2_lr_sgdp09.txt', acc_lr_cnn2)  # p=0.9
#np.savetxt('skorch_nn_cifar10_crossval_CustomCNN2_lr_sgdp05.txt', acc_lr_cnn2)  # p=0.5
#np.savetxt('skorch_nn_cifar10_crossval_CustomCNN2_lr_sgdp0.txt', acc_lr_cnn2)   # p=0


---
## Effect of different number of epochs with Adam optimizer on model accuracy

In [ ]:
# Set model architecture
model1 = model_architectures.CustomCNN1_cifar10(n_classes = 10)
model2 = model_architectures.CustomCNN2_cifar10(n_classes = 10)

n_folds = 5                        # Number of folds for the cross-validation
n_epochs_range = np.arange(1,10,2) # Range of epochs
learning_rate = 1e-3               # The learning rate.

# Initialize accuracy vector
acc_ep_cnn1 = np.zeros((len(n_epochs_range), n_folds))
acc_ep_cnn2 = np.zeros((len(n_epochs_range), n_folds))

# Loop over a range of epochs
for i in range(len(n_epochs_range)):
    # CustomCNN1
    net_temp = NeuralNetClassifier(
        module = model1,
        batch_size = 64,
        max_epochs = n_epochs_range[i],
        criterion = nn.CrossEntropyLoss,
        optimizer = optim.Adam,
        optimizer__lr = learning_rate
        )
    
    cv_results_temp = cross_validate(net_temp, train_sliceable, y_train, cv=n_folds, scoring='accuracy')
    acc_ep_cnn1[i,:] = cv_results_temp['test_score']
    
    # CustomCNN2
    net_temp = NeuralNetClassifier(
        module = model2,
        batch_size = 64,
        max_epochs = n_epochs_range[i],
        criterion = nn.CrossEntropyLoss,
        optimizer = optim.Adam,
        optimizer__lr = learning_rate
        )
    
    cv_results_temp = cross_validate(net_temp, train_sliceable, y_train, cv=n_folds, scoring='accuracy')
    acc_ep_cnn2[i,:] = cv_results_temp['test_score']
    
# Save the results
np.savetxt('skorch_nn_cifar10_crossval_CustomCNN1_epochs_adam.txt', acc_lr_cnn1)
np.savetxt('skorch_nn_cifar10_crossval_CustomCNN2_epochs_adam.txt', acc_lr_cnn2)    


## Calculate the confusion matrix
Use Adam with lr=1e-3, n_epochs=5 and the testing dataset.
Look at the cernels the CNN's create.

In [ ]:
## Set model architecture
model1 = model_architectures.CustomCNN1_cifar10(n_classes = 10)
model2 = model_architectures.CustomCNN2_cifar10(n_classes = 10)

n_epochs = 5          # Number of epochs
learning_rate = 1e-3  # The learning rate

## Define the neural networks
# CustomCNN1
net1 = NeuralNetClassifier(
    module = model1,
    batch_size = 64,
    max_epochs = n_epochs,
    criterion = nn.CrossEntropyLoss,
    optimizer = optim.Adam,
    optimizer__lr = learning_rate
)

# CustomCNN2
net2 = NeuralNetClassifier(
    module = model2,
    batch_size = 64,
    max_epochs = n_epochs,
    criterion = nn.CrossEntropyLoss,
    optimizer = optim.Adam,
    optimizer__lr = learning_rate
)

## Train the models
net1.fit(train_sliceable, y_train)
net2.fit(train_sliceable, y_train)

# Calculate predicted labels for the testing dataset
y_pred_cnn1 = net1.predict(test_sliceable)   # CustomCNN1
y_pred_cnn2 = net1.predict(test_sliceable)   # CustomCNN2

# Calculate the confusion matrix for the testing data
cm1 = confusion_matrix(y_test, y_pred_cnn1, labels=net1.classes_, normalize='all') # CustomCNN1
cm2 = confusion_matrix(y_test, y_pred_cnn2, labels=net2.classes_, normalize='all') # CustomCNN2

# Save the confusion matrix
np.savetxt('nn_cifar10_confmatrix_ccnn1.txt', cm1)
np.savetxt('nn_cifar10_confmatrix_ccnn2.txt', cm2)
